<a href="https://colab.research.google.com/github/Parkseojin2001/Hands-On-ML/blob/main/02_end_to_end_machine_learning_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**예제 프로젝트 : 부동산 관련 문제**

```
  <주요 단계>

  1. 큰 그림 보기

  2. 데이터 구하기

  3. 데이터로부터 통찰을 얻기 위해 탐색하고 시각화하기

  4. 머신러닝 알고리즘을 위해 데이터를 준비

  5. 모델을 선택하고 훈련

  6. 모델을 상세하게 조정

  7. 솔루션을 제시

  8. 시스템을 론칭하고 모니터링하고 유지 보수하기
```

# 2.1 실제 데이터로 작업하기

StatLib 저장소에 있는 캘리포니아 주택 가격 데이터셋을 사용

<img src = "https://tensorflowkorea.files.wordpress.com/2018/05/e18489e185b3e1848fe185b3e18485e185b5e186abe18489e185a3e186ba-2018-05-30-e1848be185a9e18492e185ae-11-51-37.png?w=1250">



# 2.2 큰 그림 보기

할 일 : 캘리포니아 인구조사 데이터를 사용해 캘리포니아의 주택 가격 모델 만들기

목표 : 모델을 학습시켜 다른 측정 데이터가 주어졌을 때 구역의 중간 주택 가격을 예측

## 2.2.1 문제 정의

모델의 출력(구역의 중간 주택 가격에 대한 예측)이 여러 가지 다른 신호와 함께 다른 머신러닝 시스템에 입력으로 사용된다.

<img src = "https://tensorflowkorea.files.wordpress.com/2018/05/e18489e185b3e1848fe185b3e18485e185b5e186abe18489e185a3e186ba-2018-05-31-e1848be185a9e1848ce185a5e186ab-10-42-47.png">

```
  파이프 라인(pipeline)
  - 데이터 처리 컴포넌트(component)들이 연속되어 있는 것을 의미한다.
  
  - 머신러닝 시스템은 데이터를 조작하고 변환할 일이 많아 자주 사용한다.
```

**문제 정의**
- 지도 학습, 비지도 학습, 강화 학습 중 무엇일까?

- 분류나 회귀인가 아니면 다른 어떤 작업인가?

- 배치 학습과 온라인 학습 중 어느 것을 사용해야 되나?

```
  - 레이블된 훈련 샘플이 있으므로 지도 학습 작업이다.

  - 값을 예측해야 하므로 회귀 문제(사용할 특성이 여러 개이므로 구체적으로
  다중 회귀)이며 각 구역마다 하나의 값을 예측하므로 단변량 회귀 문제이다.

  - 시스템으로 들어오는 데이터에 연속적인 흐름이 없으므로 빠르게 변하는 데이터에 적응하지 않아도 되고, 데이터가 메모리에 들어갈 만큼 충분히 작으므로 배칙 학습이다.

```

## 2.2.2 성능 측정 지표 선택

**평균 제곱 오차(RMSE - root mean sqare error)** : 회귀 문제의 전형적인 성능 지표로 오차가 커질수록 이 값은 더욱 커지므로 예측에 얼마난 많은 오류가 있는지 가늠하게 해준다.

$$RMSE(X, h) = \sqrt{\frac{1}{m}\sum_{i = 1}^m(h(x^{(i)})-y^{(i)})^2}$$

**평균 절대 오차(MAE - mean absolute error)** : 이상치로 보이는 구역이 많은 경우

$$\text{MAE}(\boldsymbol{X},h) = \frac{1}{m} \sum_{i=1}^m |h(\boldsymbol{x^{(i)}}) - y^{(i)}|$$

RMSE와 MAE의 공통점 : 예측값의 벡터와 타깃값의 벡터 사이의 거리를 재는 방법이다.


## 2.2.3 가정 검사

마지막으로 지금까지 만든 가정을 나열하고 검사해보는 것이 좋다.

```
ex. 시스템이 출력한 구역의 가격이 머신러닝 시스템의 입력으로 들어가게 되는데 이 값이 있는 그대로 사용될 거라 가정.
But, 하위 시스템에서 이 값을 ('저렴', '보통', '고가' 같은) 카테고리로 바꾸고 가격 대신 카테고리를 사용할 경우에 이는 회귀문제가 아니라 분류 문제로 바뀌는 문제가 발생
```